# Eigensystems

This notebook explores methods for computing eigenvalues and eigenvectors, i.e, finding $x$ and $\lambda$ that satisfy

$$ A x = \lambda x $$

For general $A$ this can be quite challenging so we restrict our attention to matrices $A \in \mathbb{R}^{n\times n}$.

There are a huge range of the methods for computing or estimating eigenvalues/eigenvectors. We will look at a few of the simpler ones in this course.

In [3]:
import numpy as np
from numpy import linalg as la

# A test eigensystem

For the examples below I use the following matrix as an example. As a check on our methods I will also use NumPy's built in methods to compute the eigenvalues and eigenvectors

In [142]:
A = np.array([[2, -1, 0],[-1, 2, -1],[0, -1, 2]])

eigenSystem = la.eig(A)
print("Eigenvalues =", eigenSystem[0])
print("Eigenvectors =\n", eigenSystem[1])   # The eigen vectors are the columns of this matrix

Eigenvalues = [3.41421356 2.         0.58578644]
Eigenvectors =
 [[-5.00000000e-01 -7.07106781e-01  5.00000000e-01]
 [ 7.07106781e-01  4.05925293e-16  7.07106781e-01]
 [-5.00000000e-01  7.07106781e-01  5.00000000e-01]]


In [136]:
lmaxCheck = np.max(eigenSystem[0])
lsecondCheck = np.sort(eigenSystem[0])[1]
lminCheck = np.min(eigenSystem[0])

print("The maximum eigenvalue is:", lmaxCheck)
print("The second  eigenvalue is:", lsecondCheck)
print("The minimum eigenvalue is:", lminCheck)

The maximum eigenvalue is: 3.4142135623730914
The second  eigenvalue is: 1.9999999999999998
The minimum eigenvalue is: 0.5857864376269049


# Maximum eigenvalue via the power method

Here we implemenet the power method for computing the maximum eigenvalue of a matrix and the associated eigenvector.

Also check out this [nice video](https://www.youtube.com/watch?v=yBiQh1vsCLU) showing the method in action

In [57]:
def MaxEigenvalue(A, err):
    (m,n) = A.shape
    if(m != n):
        print("Matrix must be square")
        return
    x = np.random.rand(m)
    
    lam = 0.1
    lamprev = 1
    while np.abs(1-lam/lamprev) > err:
        p = np.dot(A,x)
        lamprev = lam
        lam = np.max(p)
        x = p/lam
        
    x = x/la.norm(x)
    return (lam,x)

In [133]:
largestEigen = MaxEigenvalue(A, 1e-14)

# Check that eigenvalue and eigenvalue satisfy the eigen equation
print(np.dot(A,largestEigen[1]) - largestEigen[0] *largestEigen[1],"\n")

# Compare the eigenvalue to NumPy's result
print(largestEigen[0], largestEigen[0]- lmaxCheck,"\n")

# Compare the eigenvector to NumPy's result
print(np.abs(largestEigen[1])-np.abs(eigenSystem[1][:,0]))

[ 9.76996262e-15 -3.33066907e-14  3.75255382e-14] 

3.414213562373048 -4.3520742565306136e-14 

[ 1.02695630e-14  1.11022302e-16 -1.01585407e-14]


# Minimum eigenvalue for an invertible matrix

If the matrix in question is invertable and has a small condition number, one way to find the smallest eigenvalue of a matrix $A$ is to find the largest eigenvalue of $A^{-1}$ and take the reciprical. This method is not very useful if we are trying to calcualte a condition number via the $L^2$ norm (as we only know if the matrix can be inverted without error by computing the condition number, but we need to compute the condition number to know if it is safe to invert the matrix numerically).

Assuming we know it is safe to numerically invert the matrix, we could invert it using the GaussianElimination function we looked at in another notebook. Here I will just use NumPy's built in function.

In [127]:
lmin = 1/MaxEigenvalue(la.inv(A), 1e-14)[0]

# Compare the result to NumPy's calculation
print(lmin, lmin - lminCheck)

0.5857864376269045 -3.3306690738754696e-16


#  Minimum eigenvalue for symmetric, positive definite matrices

If we have a symmetric matrix which is positive definite then we can find the minimum eigenvalue without inverting the matrix using the algorithm below. In the lectures we proved that $A^T A$ has these properties.

If we define 

$$ B = A -\lambda_\max I$$

Then so long as $A$ is positive definite the max eigenvalue of $B$ will be the minimum eigenvalue of $A$. Thus we can find the minimum eigenvalue of $A$ by finding the maximum eigenvalue of $B$ using the power method.

This method is suitable for calculating the condition number of a matrix.

In [115]:
def MinEigenvalueSymmetricPositiveDefinite(A, err):
    
    (m,n) = A.shape
    if(m != n):
        print("Matrix must be square")
        return
    
    lmax = MaxEigenvalue(A,err)[0]
    B = A - np.identity(m)*lmax
    
    return (MaxEigenvalue(B,err)[0] + lmax, MaxEigenvalue(B,err)[1])

In [117]:
smallEigen = MinEigenvalueSymmetricPositiveDefinite(A, 1e-14)

# Check that eigenvalue and eigenvalue satisfy the eigen equation
print(np.dot(A,smallEigen[1]) - smallEigen[0] *smallEigen[1],"\n")

# Check the eigenvalue agrees with NumPy's result
print(smallEigen[0], smallEigen[0] - lminCheck)

[1.38222767e-14 1.40998324e-14 5.93969318e-15] 

0.5857864376268851 -1.9761969838327786e-14


# Hotelling's deflation: finding the second largest eigenvalue of

We've seen how to find the maximum and minimum eigenvalues. What about other eigenvalues? In general if you are looking for multiple eigenvalues there are a range of other methods. Here we look at a method for finding the second largest eigenvalue.

In this algorthm we construct the matrix

$$ B = A - \lambda_1 u_1\otimes u_1$$

where $\lambda_1$ is the largest eigenvalue, $u_1$ is the corresponding (unit normalized) eigenvector, and $\otimes$ is the outer product.

The matrix $B$ has the same eigenvectors as $A$, ans the same eigenvalues except the largest one has been replaced by 0. Thus if we use the power method to find the largest eigenvalue of $B$, this will be the second largest eigenvalue of $A$.

In [149]:
maxEigen = MaxEigenvalue(A, 1e-14)
l1 = maxEigen[0]
u1 = maxEigen[1]

In [150]:
B = A - l1*np.outer(u1, u1)

In [159]:
lsecond = MaxEigenvalue(B, 1e-12)

print(lsecond[0], lsecond[0] - lsecondCheck)

2.0000000004403478 4.4034798030168076e-10
